# Agent Loan Decisioning

In [3]:
import datetime
from datetime import date,timedelta
import numpy as np
import pandas as pd


Qualifying Matrix

1.	Agent Length on Service - 3 months

2.	Agent Frequency of use/activity - >= 65 transactions(count) in total

3.	Agent Frequency of rebalance - >= 45 inbound(count) in total

4.	Median Cash-In for last 90 days - avg. txn amnt - ghs5000 

5.	Mode Cash-In for last 90 days - lowest amnt cash in - ghs1000 

6.	Median Cash-Out for last 90 days - avg. txn amnt - ghs5000

7.	Mode Cash-Out for last 90 days - lowest amnt cash out - ghs1000 

In [4]:
#Reading data from csv file

momo_data = pd.read_csv(r'\Users\Chakdahah\Cleaned_Momo.csv', skipinitialspace = True)

# Clean whitespaces in colunn name and entire dataframe

momo_data =momo_data.rename(columns=lambda x: x.strip())
momo_data =momo_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

momo_data.head()

,TRANSACTION_DATE,TRANSACTION_TIME,FROM_ACCT,FROM_NAME,FROM_NO.,TRANS_TYPE,AMOUNT,FEES,BA_ BEFORE,BAL_AFTER,TO_NO.,TO_NAME,TO_ACCT,F_ID,REF,OVA
0,12/8/21,2:28:00 PM,43122976,Emmanuel Lamptey,244471083,DEBIT,240.0,0.0,1060,820,547169801,BluPay,57781322.0,15143397094,Ghana Telecommunicatio ns Company Limited,blupen.sp
1,12/8/21,1:05:00 PM,49853126,Interpush OVA,547169692,TRANSFER,250.0,0.0,810,1060,244471083,Emmanuel Lamptey,43122976.0,15142237113,NaN,Internal
2,12/6/21,10:19:00 AM,43122976,Emmanuel Lamptey,244471083,DEBIT,10.0,0.0,820,810,0,cis,53169694.0,15110411016,Internet Bundle,cis
3,12/5/21,5:47:00 PM,43122976,Emmanuel Lamptey,244471083,TRANSFER,21.0,0.0,841,820,596043235,Eric Kosi Oyeh,60123390.0,15103123903,1,Internal
4,12/30/21,10:55:00 AM,43122976,Emmanuel Lamptey,244471083,TRANSFER,20.0,0.0,450,430,247408068,Michael Ampadu,44181533.0,15441625272,1,Internal


In [5]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TRANSACTION_DATE  89 non-null     object 
 1   TRANSACTION_TIME  89 non-null     object 
 2   FROM_ACCT         89 non-null     int64  
 3   FROM_NAME         89 non-null     object 
 4   FROM_NO.          89 non-null     int64  
 5   TRANS_TYPE        89 non-null     object 
 6   AMOUNT            89 non-null     float64
 7   FEES              89 non-null     float64
 8   BA_ BEFORE        89 non-null     int64  
 9   BAL_AFTER         89 non-null     int64  
 10  TO_NO.            89 non-null     int64  
 11  TO_NAME           87 non-null     object 
 12  TO_ACCT           87 non-null     float64
 13  F_ID              89 non-null     int64  
 14  REF               70 non-null     object 
 15  OVA               89 non-null     object 
dtypes: float64(3), int64(6), object(7)
memory usag

In [6]:
# Set empty list and dict
masterData = []
userData= {}

# Calculation of scoring parameters

In [7]:

rows = momo_data.count()[0] # Agent Frequency of use
userData["freq_of_active"] = rows



MOMO=momo_data[(momo_data == 'CASH_IN').any(axis=1)] # frequency of rebalance
MOMOR = MOMO.count()[0]
userData["freq_of_repl"] = MOMOR



meadnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].median() #Median Cash-In for last 90days
# userData["mead_cash_in"] = meadnie
# userData["mead_cash_in"]
userData["mead_cash_in_calc"] = meadnie*4


modnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].mode() # Mode Cash-In for last 90 days
userData["modnie_cash_in_calc"] = modnie*4
userData["modnie_cash_in_calc"] = userData["modnie_cash_in_calc"]*4
userData["modnie_cash_in_calc"] = max(userData["modnie_cash_in_calc"])


meadniout = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].median() # Median Cash-Out for last 90 days
# userData["mead_cash_out"] = meadniout
# userData["mead_cash_out"]
userData["mead_cash_out_calc"] = meadniout*4


modno = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].mode() # Mode Cash-Out for last 90 days
userData["modno_cash_out_calc"] = modno*4 
userData["modno_cash_out_calc"] = userData["modno_cash_out_calc"]*4
userData["modno_cash_out_calc"] = max(userData["modno_cash_out_calc"])




# Scoring Criteria Calculation

In [8]:
user_copy = userData.copy()
masterData.append(user_copy)

In [9]:
### scoring

inelligible = []
elligible = []

In [10]:
Credit_score = 100


for index in range(len(masterData)):

    ## Frequency of use  >= 65 transactions(count) in total

    if masterData[index]["freq_of_active"]> 65 and masterData[index]["freq_of_active"] <= 75:
        user_copy = masterData[index]
#         print(user_copy)
        Credit_score += 50 
    elif masterData[index]["freq_of_active"]> 75:
        elig_copy = masterData[index]
        Credit_score+= 100

        
        
#Agent Frequency of rebalance - >= 45 inbound(count) in total

    if masterData[index]["freq_of_repl"]>= 45 and masterData[index]["freq_of_repl"] <= 54:
        user_copy = masterData[index]
        Credit_score += 50 
    elif masterData[index]["freq_of_active"]> 54:
        elig_copy = masterData[index]
        Credit_score+= 100

        
        
        # Median Cash-In for last 90 days - avg. txn amnt - ghs5000 

        
    if masterData[index]["mead_cash_in_calc"] >= 5000:
        user_copy = masterData[index]
        Credit_score += 50
        
    else:
        Credit_score+= 0

        
        
          # Mode Cash-In for last 90 days - lowest amnt cash in - ghs1000 

        
    if masterData[index]["modnie_cash_in_calc"] >= 1000:
        user_copy = masterData[index]
        Credit_score += 50 
        
    else:
        elig_copy = masterData[index]
        Credit_score+= 0
        
        
        
       # Median Cash-Out for last 90 days - avg. txn amnt - ghs5000


    if masterData[index]["mead_cash_out_calc"] > 5000:
        user_copy = masterData[index]
        Credit_score += 50 
        
    else:
        elig_copy = masterData[index]
        Credit_score+= 0
        
#     Mode Cash-Out for last 90 days - lowest amnt cash out - ghs1000 
        
    if masterData[index]["modno_cash_out_calc"] >= 1000:
        user_copy = masterData[index]
        Credit_score += 50 
        
    else:
        elig_copy = masterData[index]
        Credit_score+= 0     
    

#         print(elig_copy)
elligible.append(elig_copy)
        
        
        
        
print("Inelligble", inelligible)
print('\n')
print("Elligible", elligible)
    
    

Inelligble []


Elligible [{'freq_of_active': 89, 'freq_of_repl': 2, 'mead_cash_in_calc': 4190.0, 'modnie_cash_in_calc': 27040.0, 'mead_cash_out_calc': 2000.0, 'modno_cash_out_calc': 9600.0}]


In [11]:
type(elligible)

list

In [14]:
# from collections import OrderedDict, defaultdict

# ee = defaultdict(elligible)
elligible.to_dict(elligible)


NameError: name 'df' is not defined

In [ ]:
### convert dict to df

pd.DataFrame.from_dict(Elligible)

In [ ]:
## convert df to csv



In [ ]:
Credit_score

# Applying scores to Merchants

In [ ]:
    
    if Credit_score >= 200 and Credit_score <= 250:
        print("You qualify for a GHS 1000 loan")

    elif Credit_score >= 251 and Credit_score <= 350:
        print("You qualify for a GHS 2000 loan")

    elif Credit_score >= 351 and Credit_score <= 400:
        print("You qualify for a GHS 3000 loan")

    elif Credit_score >= 401 and Credit_score <= 500:
        print("You qualify for a GHS 5000 loan")




# Score and risk grade table 

In [ ]:
details = {
    'Risk Ranking' : ['RR-1', 'RR-2', 'RR-3', 'RR-4'],
    'Minimum Score' : [200,251,351,401],
    'Maximum Score' : [250,350,400,500],
    'Qualifying Amount(GHS)':[1000,2000,3000,5000]
}
  
# creating a Dataframe object 
df = pd.DataFrame(details)
df